# DS4420 Transit Data EDA
Exploring GTFS Schedule data for the MBTA. Loading data, observing features and preparing for analysis.

In [1]:
# imports
import pandas as pd
import requests

In [2]:

api_key = 'f8a9ad97579d4ed2978147f7187eced5'

# MBTA Alerts endpoint
url = "https://api-v3.mbta.com/alerts"

# Option 1: Sending the API key in headers
headers = {"x-api-key": api_key}
response = requests.get(url, headers=headers)

# Option 2: Alternatively, you can pass it as a query parameter
# response = requests.get(url, params={'api_key': api_key})

if response.status_code == 200:
    alerts_data = response.json()
    print(alerts_data)
else:
    print(f"Error: {response.status_code} - {response.text}")


{'data': [{'attributes': {'active_period': [{'end': '2025-03-31T02:59:00-04:00', 'start': '2025-03-24T03:00:00-04:00'}], 'banner': None, 'cause': 'UNKNOWN_CAUSE', 'created_at': '2025-03-17T13:35:48-04:00', 'description': '-The former Middleborough/Lakeville will now become the Lakeville Station, which will be used for Cape Flyer service.\r\n-The new Middleborough Station will be opening at 52 West Grove Street, Middleborough, MA 02346.\r\n-All new stations will be a Zone 8 fare ($12.25 one-way).\r\n-There will be 32 weekday and 26 weekend trips with timed transfers at East Taunton Station.\r\n\nNew stations & addresses:\r\n-East Taunton Station: 1141 County Street, Taunton, MA 02780\r\n-Freetown Station: 153 South Main Street, Freetown, MA 02702\r\n-Fall River Depot: 825 Davol Street, Fall River, MA 02720\r\n-Church Street Station: 387 Church Street, New Bedford, MA 02745\r\n-New Bedford Station: 536 Acushnet Avenue, New Bedford, MA 02740', 'duration_certainty': 'KNOWN', 'effect': 'SCH

In [11]:
import requests
from datetime import datetime

# MBTA API endpoint for alerts
url = "https://api-v3.mbta.com/alerts"

# Parameters to get alerts
params = {
    "api_key": "f8a9ad97579d4ed2978147f7187eced5",  # Replace with your API key
    "page[limit]": 100  # Adjust as needed
}

# Fetch alerts data
response = requests.get(url, params=params)
alerts_data = response.json()

# Extract 'created_at' dates from the alerts
dates = []
for alert in alerts_data.get('data', []):
    created_at = alert.get('attributes', {}).get('created_at')
    if created_at:
        dt = datetime.fromisoformat(created_at.replace('Z', '+00:00'))
        dates.append(dt)

# Compute min and max dates if available
if dates:
    min_date = min(dates)
    max_date = max(dates)
    print("Minimum alert date:", min_date)
    print("Maximum alert date:", max_date)
else:
    print("No alert dates found.")


Minimum alert date: 2020-04-07 09:38:35-04:00
Maximum alert date: 2025-03-26 19:28:39-04:00


In [19]:
# Extract 'created_at' dates from the alerts
dates = []
for alert in alerts_data.get('data', []):
    banner = alert.get('attributes', {}).get('header')
    if banner:
        dates.append(banner)

dates

['Red Line: Delays of about 15 minutes due to police action at Park St.',
 'Bowdoin station will be closed from 7 PM on Friday, April 11, through end of service on Monday, April 14. Riders can use Government Center, a four minute walk from Bowdoin. There will be longer wait times between trains throughout this work.',
 'Green Line B Branch: Service will originate / terminate at the Lake Street platform, just outside of Boston College Station, from 8:30 PM on Friday, April 11 through the end of service on Sunday, April 13. This to allow for maintenance work in the BC yard',
 'Mattapan Trolley: Shuttle buses will replace service from 11:30 PM to end of service, March 26 - 27, for track work. ',
 'Red Line Ashmont Branch: Service between JFK/UMass and Ashmont will operate with two shuttle trains from April 10 - 30 to allow for critical track work. Riders will experience longer wait times for trains and should plan extra travel time.',
 'Mattapan Trolley: Shuttle buses replace Mattapan Tro

In [13]:
import json
import requests
from datetime import datetime

# MBTA API endpoint for alerts
url = "https://api-v3.mbta.com/alerts"

# Parameters to get subway alerts
params = {
    "api_key": "f8a9ad97579d4ed2978147f7187eced5",
    "filter[route_type]": "0,1",  # 0 for subway, 1 for rail
    "filter[activity]": "BOARD,EXIT,RIDE",
    "sort": "-created_at",
    "page[limit]": 100  # Adjust as needed
}

# Fetch alerts data
response = requests.get(url, params=params)
alerts_data = response.json()

# Filter for severe weather alerts
severe_weather_alerts = []
for alert in alerts_data.get('data', []):
    # Safely get attributes as a dictionary
    attributes = alert.get('attributes') or {}
    # Get the banner text, if available
    banner = attributes.get('banner', '')
    # Use lower() to make the search case-insensitive
    if banner and 'weather' in banner.lower():
        created_at = attributes.get('created_at')
        if created_at:
            created_at_dt = datetime.fromisoformat(created_at.replace('Z', '+00:00'))
        else:
            created_at_dt = None
        severe_weather_alerts.append({
            'created_at': created_at_dt,
            'banner': banner,
            'description': attributes.get('description', 'No description available'),
            'route_id': attributes.get('route_id', 'N/A'),
            'route_type': attributes.get('route_type', 'N/A')
        })

# Remove any alerts where created_at is None before sorting
severe_weather_alerts = [alert for alert in severe_weather_alerts if alert['created_at'] is not None]
severe_weather_alerts.sort(key=lambda x: x['created_at'])

# Print results
print(f"Found {len(severe_weather_alerts)} severe weather alerts for subway")
if severe_weather_alerts:
    print("\nEarliest alert:")
    earliest = severe_weather_alerts[0]
    print(f"Date/Time: {earliest['created_at']}")
    print(f"Banner: {earliest['banner']}")
    print(f"Description: {earliest['description']}")
    print(f"Route ID: {earliest['route_id']}")
    
    print("\nLatest alert:")
    latest = severe_weather_alerts[-1]
    print(f"Date/Time: {latest['created_at']}")
    print(f"Banner: {latest['banner']}")
    print(f"Description: {latest['description']}")
    print(f"Route ID: {latest['route_id']}")
    
    print("\nAll alerts in chronological order:")
    for alert in severe_weather_alerts:
        print(f"\nDate/Time: {alert['created_at']}")
        print(f"Banner: {alert['banner']}")
        print(f"Description: {alert['description']}")
        print(f"Route ID: {alert['route_id']}")
else:
    print("No severe weather alerts found in the specified time period")


Found 0 severe weather alerts for subway
No severe weather alerts found in the specified time period
